# Impementation of Catboost

To solve this problem I descided on using CatBoost Classifier. CatBoost is a decision tree clasifier that accells on categorical features.

In [38]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

## Metadata and parameters

For the model to be able to distinguish between continous, number valriables and categorical values a list of string varibles, as well as some "categorical-looking" variables are passed in to a list. More on this choice in the other notebook.

After importing the dataset all nan values are sett to -1, a value not used in the dataset, here used to make a new category representing NaN values for the model. Afther that the dataset is split into X and Y sets, or variable and target sets.


In [39]:
categoricalVars = [0, 1, 5, 7, 9, 11, 12, 14, 15, 23, 24, 27, 2, 3, 13, 18, 20, 26]
learningRate = 0.01

addHexVars = False
addIntCategorcalVars = False
iterations = 3500
depth = 11
seed = 1
evalSize = 0.15

dataTrain = pd.read_csv("data/challenge1_train.csv")

noNans = dataTrain.fillna(0)

values = noNans.values
X = values[:,2:31]
Y = values[:,1]


## Categorical int vars

This is togglable method if one wants to use some of the number variables as categories. To do this the variables must be converted to int to be accepted by CatBoost

In [40]:
intVars = [6, 8, 22, 28]

if addIntCategorcalVars:
    for col in intVars:
        categoricalVars.append(col)
        for i, item in enumerate(X[:,col]):
            X[:,col][i] = int(item)

## Hex vars

This is a togglable method to convert hexadecimal values to decimal numbers and make them continous variables, not categorical 

In [41]:
hexVars = [2, 3, 13, 18, 20, 26]

if addHexVars:
    for col in hexVars:
        categoricalVars.remove(col)
        for i, item in enumerate(X[:,col]):
            if "E+" in str(item):
                item = int(float(str(item)))
            X[:,col][i] = int(str(item), 16)

## Initiaing the model

Parameters are passed in to the model

In [42]:
model = CatBoostClassifier(
    iterations = iterations,
    depth = depth,
    learning_rate=learningRate,
    eval_metric="AUC",
    early_stopping_rounds=100,
    loss_function='Logloss',
    verbose=100
)

## 'train_test_split' and 'fit'

'train_test_split' splits data into two sets, one for training and fitting the model and one for evaluation. 
For highest score this should be ignored and X and Y should be passed in to 'model.fit' in place of xTrain and yTrain. 
Though there is a risk of overfitting this results in a moddel trained on the most data giving the highest score in this (or in my) case. 
To be able to display a nice graph with estimation of score a eval_set that is not contained within the training set should be passed in to 'model.fit'

'model.fit' uses 'logloss' to minimize the difference between estimation and target in the trainingset.

In [43]:
xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size=evalSize, random_state=seed)
model.fit(X, Y, cat_features=categoricalVars, plot = True)
# model.fit(xTrain, yTrain, cat_features=categoricalVars, eval_set=(xTest, yTest), plot = True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	total: 236ms	remaining: 8m 51s
100:	total: 1m 43s	remaining: 36m 34s
200:	total: 3m 51s	remaining: 39m 22s
300:	total: 5m 58s	remaining: 38m 43s
400:	total: 8m 11s	remaining: 37m 46s
500:	total: 10m 10s	remaining: 35m 30s
600:	total: 12m 6s	remaining: 33m 12s
700:	total: 14m 1s	remaining: 31m
800:	total: 15m 54s	remaining: 28m 46s
900:	total: 18m 5s	remaining: 27m 4s
1000:	total: 20m 9s	remaining: 25m 9s
1100:	total: 22m 35s	remaining: 23m 34s
1200:	total: 24m 51s	remaining: 21m 42s
1300:	total: 27m 21s	remaining: 19m 57s
1400:	total: 29m 43s	remaining: 18m
1500:	total: 31m 48s	remaining: 15m 52s
1600:	total: 34m 26s	remaining: 13m 57s
1700:	total: 37m 6s	remaining: 11m 58s
1800:	total: 39m 38s	remaining: 9m 52s
1900:	total: 42m 20s	remaining: 7m 46s
2000:	total: 45m 26s	remaining: 5m 39s
2100:	total: 48m 4s	remaining: 3m 24s
2200:	total: 50m 34s	remaining: 1m 7s
2249:	total: 51m 46s	remaining: 0us


## Predicting the test data set

Before we can use the model to make predictions on the test set we need to apply the same opperations to the train set as we have done to the train set.

In [44]:
testData = pd.read_csv("data/challenge1_test.csv").fillna(0)
testData.drop(columns=testData.columns[0], axis=1, inplace=True)
testData = testData.values

if addIntCategorcalVars:
    for col in intVars:
        for i, item in enumerate(testData[:,col]):
            testData[:,col][i] = int(item)

if addHexVars:
    for col in hexVars:
        for i, item in enumerate(testData[:,col]):
            if "E+" in str(item):
                item = int(float(str(item)))
            testData[:,col][i] = int(str(item), 16)


## Predictions

Predictions made by the model is passed out as a probability distribution of the different classes the model are classifying (0, 1).

In [45]:
predictions = model.predict_proba(testData)
predictions

array([[0.72897541, 0.27102459],
       [0.88920728, 0.11079272],
       [0.87677329, 0.12322671],
       ...,
       [0.92184236, 0.07815764],
       [0.87759518, 0.12240482],
       [0.9183203 , 0.0816797 ]])

## Writing to file

Finaly the prediction (probability of the output being a 1) is written to a csv file along with the id of the row the prediction is made on

In [46]:
out = {"id": [], "target": []}
for i, prediction in enumerate(predictions):
    out["id"].append(i+50000)
    out["target"].append(prediction[1])
df = pd.DataFrame(out) 
df.to_csv('Predictions.csv', index=False) 